<a href="https://colab.research.google.com/github/amfakh/eapanicobacoba/blob/master/Scrape_Genius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sauce:

https://medium.com/analytics-vidhya/how-to-scrape-song-lyrics-a-gentle-python-tutorial-5b1d4ab351d2

https://www.promptcloud.com/blog/scraping-song-lyrics-using-python-from-genius/

In [1]:
import urllib.request
import urllib.parse
import urllib.error
import ssl
import os
import re
import requests

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [2]:
GENIUS_API_TOKEN='PXpvxuPl_XyMM02ly-gyw8bd7tyP1avUQLZVsR4ooRmhfbTDOWG8mEHez0DGygjp'

In [3]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response# Get Genius.com song url's from artist object
    
# DEMO
# request_song_url('JKT48', 10)

In [4]:
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs

In [5]:
# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [6]:
song_json = {}
song_json["Primary Artist"] = []
song_json["Song Title"] = []
song_json["Lyrics"] = []


def scrape_from_link(url):
    req = Request(url, headers = { 'User-Agent' : 'Mozilla/5.0' })
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')

    for div in soup.findAll('h1', attrs = {'class': 'header_with_cover_art-primary_info-title'}):
        song_json["Song Title"].append(div.text.strip());

    for div in soup.findAll('a', attrs = {'class': 'header_with_cover_art-primary_info-primary_artist'}):
        song_json["Primary Artist"].append(div.text.strip());    

    for div in soup.findAll('div', attrs = {'class': 'lyrics'}):
        song_json["Lyrics"].append(div.text.strip().split("\n"));


In [7]:
# scrape_from_link('https://genius.com/Jkt48-only-today-lyrics')

In [8]:
# song_json

In [10]:
import json

def write_lyrics_to_file(artist_name, song_count):

    with open(artist_name.lower() + '.json', 'w') as outfile:
        urls = request_song_url(artist_name, song_count)
        for url in urls:
            print(url)
            scrape_from_link(url)
            # print("Scrapped: " + song_json["Song Title"][-1])
        json.dump(song_json, outfile, indent = 4, ensure_ascii = False)
  
# DEMO  
write_lyrics_to_file('JKT48', 50)

Found 50 songs by JKT48
https://genius.com/Jkt48-heavy-rotation-lyrics
https://genius.com/Jkt48-fortune-cookie-yang-mencinta-fortune-cookie-in-love-lyrics
https://genius.com/Jkt48-river-lyrics
https://genius.com/Jkt48-pesawat-kertas-365-hari-lyrics
https://genius.com/Jkt48-everyday-kachuusha-lyrics
https://genius.com/Jkt48-dirimu-melody-kimi-wa-melody-lyrics
https://genius.com/Jkt48-only-today-lyrics
https://genius.com/Jkt48-baby-baby-baby-lyrics
https://genius.com/Jkt48-first-rabbit-lyrics
https://genius.com/Jkt48-sebagian-besar-kenangan-omoide-no-hotondo-lyrics
https://genius.com/Jkt48-rapsodi-lyrics
https://genius.com/Jkt48-seesaw-game-penuh-air-mata-namida-no-seesaw-game-lyrics
https://genius.com/Jkt48-kereta-kedewasaan-otona-ressha-lyrics
https://genius.com/Jkt48-so-long-lyrics
https://genius.com/Jkt48-bersama-kamu-pelangi-dan-mentari-kimi-to-niji-to-taiyou-to-lyrics
https://genius.com/Jkt48-beginner-lyrics
https://genius.com/Jkt48-high-tension-lyrics
https://genius.com/Jkt48-kita

In [12]:
import pandas as pd

In [13]:
df = pd.read_json (r'jkt48.json')

In [14]:
df

,Primary Artist,Song Title,Lyrics
0,JKT48,Heavy Rotation,"[[Intro], One! Two! Three! Four, , [Chorus 1],..."
1,JKT48,Fortune Cookie Yang Mencinta (Fortune Cookie I...,"[[Verse 1], Walaupun diri ini menyukaimu, Kamu..."
2,JKT48,"Everyday, Kachuusha","[[Verse 1], Saat mentari mulai bersinar, Lebih..."
3,JKT48,Only Today,"[[Verse 1], Bagaimana pun juga ku ingin, Ke la..."
4,JKT48,Baby! Baby! Baby!,"[[Chorus 1], I Love you baby baby baby, alihka..."
5,JKT48,First Rabbit,"[[Verse 1], Suatu hari di dalam hutan ditemuka..."
6,JKT48,Sebagian Besar Kenangan (Omoide No Hotondo),"[[Verse 1: Melody, Shanju], Dari setiap jendel..."
7,JKT48,So Long!,"[[Intro], So long (So long), Tersenyumlah, So ..."
8,JKT48,Kita Pernah Di Sini (Koko Ni Itakoto),"[[Verse 1], Ku akan terus berjalan, Di sepanja..."
9,JKT48,Sweet & Bitter,"[[Chorus], Hei hei, pilih yang mana? Cokelat, ..."
